In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')

mypf = mypf[mypf['LatestQtr'] == 0]
#mypf = mypf[mypf['v80'] == 1]

stock_n100 = mypf['Symbol'].unique()

exclude = ['ACC','AMBUJACEM','CADILAHC','GILLETTE','NESTLEIND','PGHH','SIEMENS', 'VALIANTORG','VINATIORGA', 'TATAELXSI', 'AMARAJABAT']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol']

#stock_ids

stock_ids = stock_n100
#stock_ids = ['DIVISLAB', 'HDFCBANK']
print('Stocks to be scanned:',len(stock_ids))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Stocks to be scanned: 16


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

df_latest_analysis = pd.DataFrame()

for stock_name in stock_ids: 


    print(stock_name)

    link_blueprint = "https://www.screener.in/company/"
    
    stock_list = ['3MINDIA', 'AUBANK', 'COLPAL', 'GLAND', 'LUXIND', 'WHIRLPOOL']
    
    if stock_name in stock_list:
        actual_link = link_blueprint + stock_name.upper()
    else:
        actual_link = link_blueprint + stock_name.upper()  + "/consolidated/"

    def get_list(actual_link):
        page = requests.get(actual_link)
        link_soup = BeautifulSoup(page.content, 'html.parser')

        test_list = link_soup.find(id="profit-loss").text.replace("  ", "").split('\n')

        while("" in test_list) :
            test_list.remove("")
        while("\r" in test_list) :
            test_list.remove("\r")        
        while(" \r" in test_list) :
            test_list.remove(" \r")

        return test_list

    test_list = get_list(actual_link)

    #print(len(test_list))
    n = len(test_list) 

    if n < 100:
        actual_link = link_blueprint + stock_name.upper() 
        test_list = get_list(actual_link)
        #print(len(test_list))

    q = 0

    if n > 162:
        p = 8
        q = n-162
    elif n > 120 and n <= 162:
        p = 8
    elif n < 120 and n > 100:
        p = 7   

    else:
        p = 6

    
    #print(test_list)
    if 'TTM' in test_list:
        end_year = 'TTM'
    else:
        end_year = 'Mar 2023'
        df_yoy = pd.DataFrame()      
        #print(test_list)
        s = test_list.index(end_year) +1 - test_list.index('Mar 2017')

        df_yoy['Year'] = test_list[test_list.index('Mar 2017'):test_list.index(end_year)+1]
        df_yoy['Sales'] = test_list[test_list.index('Expenses\xa0+')-s:test_list.index('Expenses\xa0+')]
        df_yoy['Profit'] = test_list[test_list.index('Tax %')-s:test_list.index('Tax %')]

        df_yoy.set_index('Year', inplace=True)
        df_yoy = df_yoy.T
        df_yoy['Stock'] = stock_name.upper() + '.NS'
        #df_yoy
        df_latest_analysis = pd.concat([df_latest_analysis, df_yoy])
    
df_latest_analysis 

3MINDIA
['Profit & Loss', 'Standalone Figures in Rs. Crores', '/', 'View Consolidated', 'Mar 2012', 'Mar 2013', 'Mar 2014', 'Mar 2015', 'Mar 2016', 'Mar 2017', 'Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Mar 2023', 'Sales\xa0+', '1,405', '1,574', '1,742', '1,840', '2,140', '2,363', '2,556', '2,809', '2,765', '2,420', '3,078', '3,733', 'Expenses\xa0+', '1,283', '1,463', '1,630', '1,636', '1,803', '1,994', '2,076', '2,301', '2,425', '2,183', '2,696', '3,174', 'Operating Profit', '122', '112', '112', '204', '337', '369', '480', '508', '340', '238', '382', '560', 'OPM %', '9%', '7%', '6%', '11%', '16%', '16%', '19%', '18%', '12%', '10%', '12%', '15%', 'Other Income\xa0+', '5', '10', '19', '14', '18', '49', '44', '33', '42', '27', '35', '64', 'Interest', '4', '10', '15', '4', '2', '2', '2', '1', '4', '2', '4', '7', 'Depreciation', '27', '36', '48', '50', '49', '47', '42', '42', '57', '60', '53', '56', 'Profit before tax', '96', '75', '69', '165', '303', '369', '480', '497',

Year,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Stock
Sales,"2,363","2,556","2,809","2,765","2,420","3,078","3,733",3MINDIA.NS
Profit,369,480,497,320,202,360,561,3MINDIA.NS
Sales,"2,876","3,944","7,305","11,416","9,926","11,258","13,293",ADANITRANS.NS
Profit,532,"1,472",840,"1,107","1,620","1,700","1,712",ADANITRANS.NS
Sales,"7,256","8,243","9,617","11,247","10,560","14,663","16,612",APOLLOHOSP.NS
Profit,222,171,373,660,221,"1,578","1,101",APOLLOHOSP.NS
Sales,"1,520","1,466","1,870","2,264",777,"1,879","3,541",IRCTC.NS
Profit,355,346,479,730,258,890,"1,354",IRCTC.NS
Sales,958,"1,078","1,209","1,664","1,938","2,273","2,368",LUXIND.NS
Profit,92,121,155,243,366,459,198,LUXIND.NS


In [ ]:
print('Stocks updated:', df_latest_analysis.shape[0]/2)
if df_latest_analysis.shape[0] > 0:
  lastestq_stocks = list(df_latest_analysis['Stock'].unique())
  len(lastestq_stocks)

Stocks updated: 7.0


In [ ]:
if df_latest_analysis.shape[0] > 0:
  #df_analysis.columns
  df_latest_analysis = df_latest_analysis.reset_index()
  df_latest_analysis.rename({'Mar 2023': 'Latest', 'index': 'Type'}, axis=1, inplace=True)
  cols = [ 'Mar 2017','Mar 2018', 'Mar 2019', 'Mar 2020', 'Mar 2021', 'Mar 2022', 'Latest']
  #df_analysis[cols] = df_analysis[cols].apply(pd.to_numeric)
  for col in cols:
      df_latest_analysis[col] = df_latest_analysis[col].str.replace(',', '').astype(float)

In [ ]:
if df_latest_analysis.shape[0] > 0:
  df_latest_analysis['Max'] = df_latest_analysis.max(axis=1)
  df_latest_analysis['Diff%'] = round((df_latest_analysis['Max'] - df_latest_analysis['Latest'])*100/df_latest_analysis['Max'],2)

<ipython-input-6-ff4f08accfa3>:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_latest_analysis['Max'] = df_latest_analysis.max(axis=1)


In [ ]:
if df_latest_analysis.shape[0] > 0:
  df_master_analysis = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100-ATH-TTM.csv')
  df_master_analysis.columns = df_latest_analysis.columns
  df_master_analysis = df_master_analysis[~df_master_analysis['Stock'].isin(lastestq_stocks)]
  df_master_analysis = pd.concat([df_master_analysis, df_latest_analysis])
  df_master_analysis.to_csv('/content/drive/My Drive/data/stocks/Nifty100-ATH-TTM.csv', index=False)

In [ ]:
#df_latest_analysis.to_csv('/content/drive/My Drive/data/stocks/Nifty100-ATH-TTM.csv', index=False)

In [ ]:
if df_latest_analysis.shape[0] > 0:
  df_tmp = df_latest_analysis[df_latest_analysis['Diff%'] <= 5]
  tmp = pd.DataFrame(df_tmp['Stock'].value_counts())
  len(tmp[tmp['Stock']>1].index)
  star_stocks = list(tmp[tmp['Stock']>1].index)
  print(len(star_stocks))

3


In [ ]:
mypf = pd.read_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv')
mypf.loc[mypf['Stock'].isin(star_stocks), 'StarStock'] = 1
mypf.loc[mypf['Stock'].isin(lastestq_stocks), 'LatestQtr'] = 1
mypf.to_csv('/content/drive/My Drive/data/stocks/Nifty100Scrips.csv', index=False)